### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [2]:
Total_Players = purchase_data['SN'].count()  #780
Total_Players

780

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
Unique_Items = purchase_data['Item ID'].nunique()  #183
Total_Revenue = purchase_data['Price'].sum()       #2379.77
Average_Price = Total_Revenue / Total_Players      #3.05

#New Data List
data = [{
        'Unique Items': Unique_Items, 'Average Price': Average_Price, 'Number of Purchases': Total_Players,
         'Total Revenue': Total_Revenue
       }] 

summary_data = pd.DataFrame(data)
summary_data["Average Price"] = summary_data["Average Price"].map("${:.2f}".format)
summary_data["Total Revenue"] = summary_data["Total Revenue"].map("${:.2f}".format)
summary_data.head()

,Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
#1 Get frequency of unique member and gender
df = purchase_data.groupby(['Gender', 'SN']).size().reset_index(name='Freq')
#2 Get totals by Gender
df2 = df.groupby(['Gender']).count()
#3 picked up an extra column lets remove it - not sure why this happened
df2.columns = ['drop', 'Total Count'] 
df3 = df2.drop(['drop'], axis=1)

#4 Get the total percentage for each gender
df4 = df2.apply(lambda x :   100 * x / float(x.sum()))
#3 picked up an extra column lets remove it - not sure why this happened
df4.columns = ['drop2', 'Percentage of Players'] 
df5 =  df4.drop(['drop2'], axis=1)
#Concat both data sets into one and then reformat the percentage to 2dp and sort by Total Count

result = pd.concat([df3,df5], axis =1, ignore_index=False).sort_values(by='Total Count', ascending=False, na_position='first')
result['Percentage of Players'] = result['Percentage of Players'].map('${:,.2f}'.format)


result

,Total Count,Percentage of Players
Gender,,
Male,484,$84.03
Female,81,$14.06
Other / Non-Disclosed,11,$1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
#Purchase Count Totals and Rename Column
Purchase_Count_df = purchase_data.groupby(['Gender'])['Purchase ID'].agg([np.size]).reset_index()
Purchase_Count_df.columns = ['Gender', 'Purchase Count'] 

Tot_Pur_Price_df = purchase_data.groupby(['Gender'])['Price'].agg([np.sum]).reset_index()
Tot_Pur_Price_df.columns = ['Gender', 'Total Purchase Value'] 

result = pd.concat([Purchase_Count_df,Tot_Pur_Price_df], axis =1, ignore_index=False)
result['Average Purchase Price'] = result['Total Purchase Value'] / result['Purchase Count']

result.set_index(['Gender'], inplace = True) 

result2 = pd.merge(left=Purchase_Count_df,right=Tot_Pur_Price_df, left_on='Gender', right_on='Gender')
result2['Average Purchase Price'] = result2['Total Purchase Value'] / result2['Purchase Count']

#2 Get totals by Gender

Count_by_Gender = purchase_data.groupby(['Gender', 'SN']).size().reset_index(name='Freq')
Count_by_Gender = Count_by_Gender.groupby(['Gender']).count()


Summary = pd.merge(left=result2,right=Count_by_Gender, left_on='Gender', right_on='Gender')
Summary['Average Total Purchase per Person'] = Summary['Total Purchase Value'] / Summary['Freq']
#result3['Avg Pur Price'] = result2['Total Purchase Value'] / result2['Purchase Count']
Summary.set_index(['Gender'], inplace = True) 
Summary['Average Total Purchase per Person'] = Summary['Average Total Purchase per Person'].map('${:,.2f}'.format)
Summary['Average Purchase Price'] = Summary['Average Purchase Price'].map('${:,.2f}'.format)
Summary['Total Purchase Value'] = Summary['Total Purchase Value'].map('${:,.2f}'.format)
Summary = Summary.drop(['SN', 'Freq'], axis=1)
#Reogranize Columns
Summary = Summary[['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Average Total Purchase per Person']]
Summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
purchase_data_new = purchase_data.groupby(['SN', 'Age']).size().reset_index(name='Freq')

bins = [0, 10, 15, 19, 24, 29,34, 39, 200]

# Create labels for these bins
group_labels = ["< 10", "10-14", "15-19", "20-24", "25-29", "30-34",
                "35-39", "40+"]

pd.cut(purchase_data_new["Age"], bins, labels=group_labels).head()

purchase_data_new["Age Bracket"] = pd.cut(purchase_data_new["Age"], bins, labels=group_labels)

purchase_data_new = purchase_data_new.groupby(['Age Bracket']).size().reset_index(name='Freq')
Total_Unique_Members = sum(purchase_data_new["Freq"])

purchase_data_new["Percentage of Players"] = purchase_data_new["Freq"] / Total_Unique_Members *100

purchase_data_new["Percentage of Players"] = purchase_data_new["Percentage of Players"].map('{:,.2f}%'.format)
purchase_data_new.set_index(['Age Bracket'], inplace = True) 

#Rename columns
purchase_data_new.columns = ['Total Count', 'Percentage of Players'] 

purchase_data_new.head(10)

,Total Count,Percentage of Players
Age Bracket,,
< 10,24,4.17%
10-14,41,7.12%
15-19,81,14.06%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
purchase_analysis_df = purchase_data.groupby(['Age'])['Purchase ID'].agg([np.size]).reset_index()
purchase_analysis_df.columns = ['Age', 'Purchase Count']

Tot_Price_df         = purchase_data.groupby(['Age'])['Price'].agg([np.sum]).reset_index()
Tot_Price_df.columns = ['Age', 'Total Purchase Value']

#Total Unique Members by Age
Total_Unq_Mbrs = purchase_data.groupby(['SN', 'Age']).size().reset_index(name='Freq')


summary_df = pd.merge(left=purchase_analysis_df,right=Tot_Price_df, how='inner', on=['Age'])

#bin time
bins = [0, 10, 15, 19, 24, 29,34, 39, 200]

# Create labels for these bins
group_labels = ["< 10", "10-14", "15-19", "20-24", "25-29", "30-34","35-39", "40+"]

pd.cut(summary_df["Age"], bins, labels=group_labels).head()

summary_df["Age Bracket"] = pd.cut(summary_df["Age"], bins, labels=group_labels)

summary_df = summary_df.groupby(['Age Bracket']).sum()
summary_df["Average Purchase Price"] = summary_df["Total Purchase Value"] / summary_df["Purchase Count"]

#Merge previous dataframe to calculate per person
summary_df2 = pd.merge(left=summary_df,right=purchase_data_new, how='inner', on=['Age Bracket'])
summary_df2["Avg Total per Person"] = summary_df2["Total Purchase Value"] / summary_df2["Total Count"]
#Reformate values
summary_df2["Avg Total per Person"] = summary_df2['Avg Total per Person'].map('${:,.2f}'.format)
summary_df2["Average Purchase Price"] = summary_df2['Average Purchase Price'].map('${:,.2f}'.format)
summary_df2["Total Purchase Value"] = summary_df2["Total Purchase Value"].map('${:,.2f}'.format)
#Output
summary_df2[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total per Person"]]


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total per Person
Age Bracket,,,,
< 10,32,$3.41,$108.96,$4.54
10-14,54,$2.90,$156.60,$3.82
15-19,101,$3.04,$307.24,$3.79
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
Member_Sales_df = purchase_data.groupby(['SN'])['Purchase ID'].agg([np.size]).reset_index()
Member_Sales_df.columns = ['SN', 'Purchase Count']

Tot_Member_Sales_df         = purchase_data.groupby(['SN'])['Price'].agg([np.sum]).reset_index()
Tot_Member_Sales_df.columns = ['SN', 'Total Purchase Value']


Top_Spenders_df = pd.merge(left=Member_Sales_df,right=Tot_Member_Sales_df, how='inner', on=['SN'])
Top_Spenders_df ["Average Purchase Price"] = Top_Spenders_df["Total Purchase Value"] / Top_Spenders_df["Purchase Count"]
#Reformat
Top_Spenders_df["Average Purchase Price"] = Top_Spenders_df['Average Purchase Price'].map('${:,.2f}'.format)
Top_Spenders_df["Total Purchase Value"] = Top_Spenders_df["Total Purchase Value"].map('${:,.2f}'.format)


Final_Top_Spenders = Top_Spenders_df.sort_values('Purchase Count', ascending=False)
#Reorder Columns
Final_Top_Spenders[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

Final_Top_Spenders.head(5)

,SN,Purchase Count,Total Purchase Value,Average Purchase Price
360,Lisosia93,5,$18.96,$3.79
275,Iral74,4,$13.62,$3.40
246,Idastidru52,4,$15.45,$3.86
75,Asur53,3,$7.44,$2.48
274,Inguron55,3,$11.11,$3.70


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# Retrieve the Item ID, Item Name, and Item Price columns
Item_Sales_df = purchase_data[['Item ID', 'Item Name', 'Price']]

Tot_Item_Sales_Price_df  = Item_Sales_df.groupby(['Item ID', 'Item Name'])['Price'].agg([np.sum])
Tot_Item_Sales_Cnt_df  = Item_Sales_df.groupby(['Item ID', 'Item Name'])['Item ID'].agg([np.size])

Most_Popular_df2 = pd.merge(left=Tot_Item_Sales_Price_df ,right=Tot_Item_Sales_Cnt_df , how='inner', on=['Item ID', 'Item Name'])

Most_Popular_df2 = Most_Popular_df2.sort_values('size', ascending=False)

Most_Popular_df2["Item Price"] = Most_Popular_df2["sum"] / Most_Popular_df2["size"]


#Reorder Columns and Names
Most_Popular_df2.columns = ['Total Purchase Value', 'Purchase Count', 'Item Price']

#Reformat Currenct
Most_Popular_df2["Total Purchase Value"] = Most_Popular_df2['Total Purchase Value'].map('${:,.2f}'.format)
Most_Popular_df2["Item Price"] = Most_Popular_df2["Item Price"].map('${:,.2f}'.format)

Most_Pop = Most_Popular_df2[[ "Purchase Count", "Item Price", "Total Purchase Value"]]
Most_Pop.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
#Convert $ to float and remove string
Most_Pop['Total Purchase Value'] = Most_Pop['Total Purchase Value'].str.replace('$', '').astype(float)
Most_Pop2 = Most_Pop.sort_values('Total Purchase Value', ascending=False)
Most_Pop2.head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,50.76
82,Nirvana,9,$4.90,44.10
145,Fiery Glass Crusader,9,$4.58,41.22
92,Final Critic,8,$4.88,39.04
103,Singed Scalpel,8,$4.35,34.80
